In [1]:
from smach_utils2 import *
import matplotlib.pyplot as plt
#arm = moveit_commander.MoveGroupCommander('arm')


In [2]:
arm=moveit_commander.MoveGroupCommander('arm')

[ WARN] [1713205155.611092740]: Link hand_l_finger_vacuum_frame has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1713205155.615642848]: Link head_l_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1713205155.615690805]: Link head_r_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1713205155.617154137]: Group state 'neutral' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WARN] [1713205155.617207201]: Group state 'go' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WARN] [1713205156.093490294]: Kinematics solver doesn't support #attempts any

In [ ]:

whole_body=moveit_commander.MoveGroupCommander('whole_body_weighted')

In [ ]:
res = omni_base.move_base(known_location='place_shelf', time_out=200)
high_shelf_place=np.asarray([0.15050079265248406,
 -0.5861087653247212,
 0.0850017715788356,
 -0.9342371117035135,
 0.022035143687695324,
 0.0])
mid_shelf_place=np.asarray([0.2556342351169577,
 -1.6040815280188854,
 -0.008078345343693893,
 -0.05795663028496589,
 0.0019971082238159354,
 0.0])

low_shelf_place=np.asarray([0.04577467546491997,
 -2.262541528018885,
 0.00010865465630605087,
 0.701618369715034,
 -0.0003068917761841483,
 0.0]
)

In [ ]:
head.set_joint_values([-0.5,-0.65])
arm.set_named_target('go')
arm.go()

In [ ]:

rospy.sleep(0.6)
rospy.sleep(0.6)
image= cv2.cvtColor(rgbd.get_image(), cv2.COLOR_RGB2BGR)
img_msg  = bridge.cv2_to_imgmsg(image)
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
debug_image=   cv2.cvtColor(bridge.imgmsg_to_cv2(res.debug_image.image_msgs[0]), cv2.COLOR_RGB2BGR)

In [ ]:

plt.imshow(debug_image)

In [ ]:
request= segmentation_server.request_class() 

print ('#############Finding placing in plane####################',request.height.data)
#request.height.data=-1
request.height.data=0.38  #MID SHELF FOR PLACING 
area_name_numbered= 'mid_shelf'


res=placing_finder_server.call(request)
#succ=seg_res_tf(res)
print (f'Placing Area at {res.poses.data}')
tf_man.pub_static_tf(pos=[res.poses.data[0], res.poses.data[1],res.poses.data[2]], rot =[0,0,0,1], point_name='placing_area')
img=bridge.imgmsg_to_cv2(res.im_out.image_msgs[0])
#cv2.imshow('our of res'  , img)
plt.imshow(img)

In [ ]:
res = omni_base.move_base(known_location='place_shelf', time_out=12)


In [ ]:
base_grasp_D(tf_name='placing_area',d_x=0.76, d_y=0.0,timeout=30)
arm.go (mid_shelf_place)
base_grasp_D(tf_name='placing_area',d_x=0.6, d_y=0.0,timeout=30)
gripper.open()

In [ ]:
arm.go (mid_shelf_place)

In [ ]:
base_grasp_D(tf_name='placing_area',d_x=0.6, d_y=0.0,timeout=30)
gripper.open()

In [ ]:

for i in range(len(res.poses)):
    tf_man.getTF("head_rgbd_sensor_rgb_frame")
    tf_man.pub_static_tf(pos=[res.poses[i].position.x ,res.poses[i].position.y,res.poses[i].position.z], rot=[0,0,0,1],ref="head_rgbd_sensor_rgb_frame",point_name=res.names[i].data[4:] )   
    rospy.sleep(0.3)
    tf_man.change_ref_frame_tf(res.names[i].data[4:])



In [ ]:
omni_base.move_d_to(target_distance=0.78,target_link='apple')

In [ ]:
target_object='apple'
trans,_=tf_man.getTF(target_object,ref_frame="base_link")
trans


In [ ]:
target_object='apple'
_,rot= tf_man.getTF("base_link",ref_frame='map')

original_rot=tf.transformations.euler_from_quaternion(rot)[2]
trans,_=tf_man.getTF(target_object,ref_frame="base_link")


trans , original_rot

In [ ]:
_,rot= tf_man.getTF("base_link",ref_frame='map')
original_rot=tf.transformations.euler_from_quaternion(rot)[2]
target_object='apple'


succ = False
            
while not succ:
    
    _,rot= tf_man.getTF("base_link",ref_frame='map')
    trans,_=tf_man.getTF(target_object,ref_frame="base_link")

    trans
    eX, eY, eZ = trans
    
    eX+= -0.4
    eY+= -.06
    
    eT= tf.transformations.euler_from_quaternion(rot)[2] - original_rot #Original 
    print (eT)
    if eT > np.pi: eT=-2*np.pi+eT
    if eT < -np.pi: eT= 2*np.pi+eT
    rospy.loginfo("error: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(eX, eY , eT,target_object))
    X, Y, Z = trans
    rospy.loginfo("Pose: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(X, Y , eT,target_object))
    
    if abs(eX) <=0.05 :
        print ('here')
        eX = 0
    if abs(eY) <=0.05  :
        eY = 0
    if abs(eT   ) < 0.1:
        eT = 0
    succ =  eX == 0 and eY == 0 and eT==0

    
    omni_base.tiny_move( velX=0.2*+eX,velY=0.3*eY, velT=-eT,std_time=0.2, MAX_VEL=0.3) 


In [ ]:
res.names

In [ ]:
hand_cam= HAND_RGB()


In [ ]:
#clear_octo_client()

pickup_pose=[0.65,-1.2,0.0,-1.9, 0.0, 0.0]
succ= arm.go(pickup_pose)
gripper.open()


In [ ]:

image= cv2.cvtColor(hand_cam.get_image(), cv2.COLOR_RGB2BGR)
img_msg  = bridge.cv2_to_imgmsg(image)
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
debug_image=   cv2.cvtColor(bridge.imgmsg_to_cv2(res.debug_image.image_msgs[0]), cv2.COLOR_RGB2BGR)
for i in range(len(res.names)):
    if res.names[i].data== '013_apple' or res.names[i].data== '017_orange':
        bbM=res.pt_min.data[2*i:2*i+2]
        bbm=res.pt_max.data[2*i:2*i+2]
        bbx=[bbM,bbm]
print(bbx)
plt.imshow(debug_image)

# [(297, 191), (231, 226)]  at pkcp pose #Real Tak
### [(323, 266), (280, 215)]  at pkcp pose #Gazebo


In [ ]:
eX,eY= 0.3,0.3 

while abs(eX) > 0.05 or abs(eY) > 0.05:
    image= cv2.cvtColor(hand_cam.get_image(), cv2.COLOR_RGB2BGR)
    img_msg  = bridge.cv2_to_imgmsg(image)
    req      = classify_client.request_class()
    req.in_.image_msgs.append(img_msg)
    res      = classify_client(req)


    for i in range(len(res.names)):
        if res.names[i].data== '013_apple' or res.names[i].data== '017_orange':
            bbM=res.pt_min.data[2*i:2*i+2]
            bbm=res.pt_max.data[2*i:2*i+2]
            bbx=[bbM,bbm]
    print(bbx)
    eY=(bbx[1][1]-215)/215    #experience value
    #print(f'error relative to gripper cam {eY}')


    #eX=(bbx[0][0]-323)/323
    eY=(bbx[1][1]-239)/239
    eX=(bbx[0][0]-312)/312
    #eY=(bbx[1][1]-215)/215    #experience value

    print(f'error relative to gripper cam ex={eX} ey={eY}')
    omni_base.tiny_move(velX=0.5*eX, velY=0.5*-eY, MAX_VEL=0.05) 
print(f'error relative to gripper cam ex={eX} ey={eY}:::::: Error within tolerance')

In [ ]:
bbx

In [ ]:
eY=(bbx[1][1]-239)/239
eX=(bbx[0][0]-312)/312
print(f'error relative to gripper cam ex={eX} ey={eY}')


In [ ]:
#omni_base.tiny_move(velX=0.5*eX, velY=0.5*-eY, MAX_VEL=0.05) 


In [ ]:
clear_octo_client()

av=arm.get_current_joint_values()
print (av,'av')
av[0]+= -0.17###LAB
#av[0]+= -0.35###GAZ
succ=arm.go(av)
rospy.sleep(0.5)
gripper.close(force=0.06)

In [ ]:
av=arm.get_current_joint_values()
print (av,'av')
av[0]+= 0.15
succ=arm.go(av)
omni_base.tiny_move(velX=-0.3, std_time=4.0)
arm.set_named_target('go')
arm.go()
#tf_man.pub_static_tf(point_name='apple')

In [ ]:
plt.imshow(hand_cam.get_image())

In [ ]:
image= cv2.cvtColor(hand_cam.get_image(), cv2.COLOR_RGB2BGR)
img_msg  = bridge.cv2_to_imgmsg(image)
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
debug_image=   cv2.cvtColor(bridge.imgmsg_to_cv2(res.debug_image.image_msgs[0]), cv2.COLOR_RGB2BGR)
plt.imshow(debug_image)

In [ ]:
gripper.open()

In [ ]:
gripper.close(force=0.06)

In [ ]:

for i in range(len(res.poses)):
    tf_man.getTF("head_rgbd_sensor_rgb_frame")
    position = [res.poses[i].position.x ,res.poses[i].position.y,res.poses[i].position.z]
    tf_man.pub_static_tf(pos= position, rot=[0,0,0,1], ref="head_rgbd_sensor_rgb_frame", point_name=res.names[i].data[4:] )   
    rospy.sleep(0.3)
    tf_man.change_ref_frame_tf(res.names[i].data[4:])




In [ ]:
def detect_object_yolo(object_name,res):
    # find object_name in the response message from object_classification service (Yolo)
    objs=[]
    for i,name in enumerate(res.names):
        objs.append(name.data[4:])
        if name.data[4:]==object_name:return res.poses[i]
    if object_name=='all':
        
        return objs
    return []

In [ ]:
#img_msg  = bridge.cv2_to_imgmsg(rgbd.get_image())  #REAL
img_msg  = bridge.cv2_to_imgmsg(cv2.cvtColor(rgbd.get_image(), cv2.COLOR_RGB2BGR ))#GAZEB
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
objects=detect_object_yolo('all',res)   
if len (objects)!=0:
    for i in range(len(res.poses)):
        tf_man.getTF("head_rgbd_sensor_rgb_frame")
        position = [res.poses[i].position.x ,res.poses[i].position.y,res.poses[i].position.z]
        tf_man.pub_static_tf(pos= position, rot=[0,0,0,1], ref="head_rgbd_sensor_rgb_frame", point_name=res.names[i].data[4:] )   
        rospy.sleep(0.3)
        tf_man.change_ref_frame_tf(res.names[i].data[4:])
        rospy.sleep(0.3)
        pose , _=tf_man.getTF(res.names[i].data[4:])
        new_row = {'x': pose[0], 'y': pose[1], 'z': pose[2], 'obj_name': res.names[i].data[4:]}
        objs.loc[len(objs)] = new_row
objs

In [ ]:
objs[['x','y','z']].values

In [ ]:
#get closest object
rob_pos,_=tf_man.getTF('base_link')
np.argmin(np.linalg.norm(rob_pos-objs[['x','y','z']].values  , axis=1))
objs.iloc[np.argmin(np.linalg.norm(rob_pos-objs[['x','y','z']].values  , axis=1))]['obj_name']

In [ ]:
objs[objs['obj_name']==objs.obj_name.values[0]].x.values[0]

# PLANE 1
#### 7.883384	5.948245	0.701945,
#### 8.214565	5.962619	0.700034,
#### 8.213697	6.107727	0.709487,
####  7.893623	6.082935	0.693285

# PLANE 2
#### 7.88076	6.086619	0.416096,
#### 8.214128	5.9663	0.416672

# Plane 3
#### 8.20238	5.963262	0.117541

In [ ]:
head.set_named_target('neutral')

av=arm.get_current_joint_values()
av[0]=0.5
av[1]=-0.5
arm.go(av)
head.set_joint_values([-np.pi/2 , -0.5])

In [ ]:
head.set_joint_values([-np.pi/2 , -0.5])

In [ ]:


av=arm.get_current_joint_values()
av[0]=0.05

arm.go(av)

In [ ]:
head.set_joint_values([-np.pi/2 , -0.8])

av=arm.get_current_joint_values()
av[0]=0.0
av[1]=-0.5
arm.go(av)

In [ ]:
import pandas as pd


In [ ]:
import pandas as pd
objs = pd.read_csv ('/home/roboworks/Documents/objs.csv')
objs=objs.drop(columns='Unnamed: 0')

In [ ]:
objs=objs[objs['obj_name'] != 'bowl']

In [ ]:
objs

In [ ]:
objs[objs['obj_name'] != 'bowl']

In [ ]:
objs

In [ ]:
from sklearn.cluster import KMeans
shelf_objs=objs[objs['shelves']==True ]
sh_o_btm=shelf_objs[shelf_objs['z']<0.2]


In [ ]:
'other' in sh_o_top['category'].values

In [ ]:
shelf_objs=objs[objs['shelves']==True ]

sh_o_btm=shelf_objs[shelf_objs['z']<0.2]
sh_o_mdl=shelf_objs[(shelf_objs['z']>0.2 )& (shelf_objs['z']<0.5) ]
sh_o_top=shelf_objs[(shelf_objs['z']>0.5 )& (shelf_objs['z']<1.5) ]
sh_o_btm['category'].value_counts(),sh_o_mdl['category'].value_counts() 

a= sh_o_btm['category'].value_counts()
if 'other' in sh_o_btm['category'].values:a.drop('other', inplace=True)
print(f'btm shelf category {a.index[a.argmax()]}')

a= sh_o_mdl['category'].value_counts()
if 'other' in sh_o_mdl['category'].values:a.drop('other', inplace=True)
print(f'middle shelf category {a.index[a.argmax()]}')


a= sh_o_top['category'].value_counts()
if 'other' in sh_o_top['category'].values:a.drop('other', inplace=True)
print(f'top shelf category {a.index[a.argmax()]}')


In [ ]:
a

In [ ]:
kitchen =['bowl','spatula','spoon', 'bowl']
tools=['extra_large_clamp','large_clamp','small_clamp','medium_clamp','adjustable_wrench',
'flat_screwdriver','phillips_screwdriver','wood_block']
balls= ['softball','tennis_ball']
fruits= ['apple','banana', 'lemon','pear']
food =['chips_can'
,'mustard_bottle'
,'potted_meat_can'
,'tomato_soup_can'
,'tuna_fish_can']

def categorize_objs(name):
    if name in kitchen: return 'kitchen'
    elif name in tools: return 'tools'
    elif name in balls: return 'balls'
    elif name in fruits: return 'fruits'
    elif name in food: return 'food'
    return 'other'
cats=[]
for name in objs['obj_name']:
    cats.append(categorize_objs(name))
objs['category'] = cats   


In [ ]:
def categorize_objs(name):
    if name in kitchen: return 'kitchen'
    elif name in tools: return 'tools'
    elif name in balls: return 'balls'
    elif name in fruits: return 'fruits'
    elif name in food: return 'food'
    return 'other'
    
    

In [ ]:
cats=[]
for name in objs['obj_name']:
    cats.append(categorize_objs(name))
objs['category'] = cats

In [ ]:
objs

In [ ]:
import numpy as np
regions={'shelves':np.load('/home/roboworks/Documents/shelf_sim.npy'),'pickup':np.load('/home/roboworks/Documents/pickup_sim.npy')}
def is_inside(x,y):return ((area_box[:,1].max() > y) and (area_box[:,1].min() < y)) and ((area_box[:,0].max() > x) and (area_box[0,0].min() < x)) 
for name in regions:
    in_region=[]
    area_box=regions[name]
    print (area_box,name)
    for index, row in objs[['x','y','z']].iterrows():
        in_region.append(is_inside(row.x, row.y))
        print (row.x, row.y,is_inside(row.x, row.y))
    objs[name]=pd.Series(in_region)
objs

In [ ]:
(area_box[:,1].max() > row.y),(area_box[:,1].min() < row.y),((area_box[:,0].max() > row.x) , (area_box[0,0].min() < row.x)) 

In [ ]:
def is_inside(x,y):return ((area_box[:,1].max() > y) and (area_box[:,1].min() < y)) and ((area_box[:,0].max() > x) and (area_box[0,0].min() < x)) 

In [ ]:
pickup_region=np.zeros((2,2))
#np.save('/home/roboworks/Documents/pickup_region.npy', pickup_region)

In [ ]:
ab[0,0],ab[0,1]=6,6
ab[1,0],ab[1,1]=7,4.0

In [ ]:
import numpy as np

#ab=np.load('/home/roboworks/Documents/pickup_region.npy')
ab= np.load('/home/roboworks/Documents/pickup_sim.npy')

In [ ]:

ab[0,0]=5.8
ab

In [ ]:
np.save('/home/roboworks/Documents/pickup_region.npy',ab)


In [ ]:
#np.save('/home/roboworks/Documents/shelf_sim.npy',ab)
np.save('/home/roboworks/Documents/pickup_sim.npy',ab)

In [ ]:
objs[objs['pickup']==True][['x','y','z']].values

In [ ]:
rob_pos,_=tf_man.getTF('base_link')
ix=np.argmin(np.linalg.norm(rob_pos-objs[objs['pickup']==True][['x','y','z']].values  , axis=1))
ix


In [ ]:
rob_pos,_=tf_man.getTF('base_link')
ix=np.argmin(np.linalg.norm(rob_pos-objs[objs['pickup']==True][['x','y','z']].values  , axis=1))
objs[objs['pickup']==True][['obj_name','category']].iloc[ix]


In [ ]:
ix=np.argmin(np.linalg.norm(rob_pos-objs[objs['pickup']==True][['x','y','z']].values  , axis=1))

In [ ]:
pose=np.asarray((8.212561, 6.028789, 0.12))
pose

In [ ]:
tf_man.pub_static_tf(pos=pose,point_name='PLACING')

In [ ]:
#pose=np.asarray((8.212561, 6.028789, 0.12))
pose=np.asarray((8.062455, 6.0622816, 0.6610926))
tf_man.pub_static_tf(pos=pose,point_name='PLACING')

In [ ]:
arm['arm_lift_joint']=0.0

In [ ]:
omni_base.move_base(known_location='shelf')

In [ ]:
omni_base.tiny_move(velT=-0.7,std_time=1)

In [ ]:
av=arm.get_current_joint_values()
high_shelf_place=av
high_shelf_place


In [ ]:
place_high_shelf=np.asarray([0.15050079265248406,
 -0.5861087653247212,
 0.0850017715788356,
 -0.9342371117035135,
 0.022035143687695324,
 0.0])

In [ ]:
arm.go(place_high_shelf)

av=arm.get_current_joint_values()


In [ ]:
av=arm.get_current_joint_values()
high_shelf_place=av

In [ ]:
arm.go(high_shelf_place)

In [ ]:
av=arm.get_current_joint_values()
mid_shelf_place=av
mid_shelf_place

In [ ]:
av=arm.get_current_joint_values()
low_shelf_place=av
low_shelf_place

In [ ]:
high_shelf_place,low_shelf_place,mid_shelf_place

In [ ]:
high_shelf_place=np.asarray([0.4337121304669095,
 -1.3139266720690426,
 0.08621383272112926,
 -0.4260850221072081,
 0.02285051353933021,
 0.0])

In [ ]:
low_shelf_place=np.asarray([-5.465250513158978e-07,
 -1.8875162206087301,
 0.08646571240675005,
 0.3040738848075222,
 0.022992974203400962,
 0.0]
)

In [ ]:
high_shelf_place=np.asarray([0.15050079265248406,
 -0.5861087653247212,
 0.0850017715788356,
 -0.9342371117035135,
 0.022035143687695324,
 0.0])
mid_shelf_place=np.asarray([0.2556342351169577,
 -1.6040815280188854,
 -0.008078345343693893,
 -0.05795663028496589,
 0.0019971082238159354,
 0.0])

low_shelf_place=np.asarray([0.04577467546491997,
 -2.262541528018885,
 0.00010865465630605087,
 0.701618369715034,
 -0.0003068917761841483,
 0.0]
)

In [ ]:
tf_man.getTF('placing_area_top_shelf1')

In [ ]:
request= segmentation_server.request_class() 
#r = cv2.getTrackbarPos('Plane height (cms)', 'class rgbd')
#request.height.data=0.9
print ('#############Finding placing in plane####################',request.height.data)
request.height.data=-1
#head.set_joint_values([ 0.1, -0.5])
res=placing_finder_server.call(request)
#succ=seg_res_tf(res)
print (f'Placing Area at {res.poses.data}')
tf_man.pub_static_tf(pos=[res.poses.data[0], res.poses.data[1],res.poses.data[2]], rot =[0,0,0,1], point_name='placing_area')
img=bridge.imgmsg_to_cv2(res.im_out.image_msgs[0])
#cv2.imshow('our of res'  , img)
plt.imshow(img)

In [ ]:
tf_man.getTF('placing_area')

In [ ]:
head.set_named_target('neutral')

In [ ]:
arm.set_named_target('go')
arm.go()

In [ ]:
res = omni_base.move_base(known_location='place_shelf', time_out=10)

In [ ]:
high_shelf_place=[0.65,         -1.139,         0.08621,        -0.4260,        0.02285,         0.0]#REAL
high_shelf_place=[0.4337,         -1.3139,         0.08621,        -0.4260,        0.02285,         0.0] #GAZ

In [ ]:
arm.set_joint_value_target(high_shelf_place)
arm.go()

In [ ]:
def base_grasp_D(tf_name,d_x=0.66,d_y=-0.1,timeout=1.0):
    timeout = rospy.Time.now().to_sec() + timeout
    rob_pos,rot=tf_man.getTF('base_link')    
    original_rot=tf.transformations.euler_from_quaternion(rot)[2]
    succ = False 
    target_object= tf_name        
    while (timeout >= rospy.Time.now().to_sec()) and not succ:            
        _,rot= tf_man.getTF("base_link",ref_frame='map')
        trans,_=tf_man.getTF(target_object,ref_frame="base_link")
        #trans
        eX, eY, eZ = trans
        eX+= -d_x  #x offest
        eY+= -d_y #y Offset
        eT= tf.transformations.euler_from_quaternion(rot)[2] - original_rot #Original 
        print (eT)
        if eT > np.pi: eT=-2*np.pi+eT
        if eT < -np.pi: eT= 2*np.pi+eT
        rospy.loginfo("error: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(eX, eY , eT,target_object))
        X, Y, Z = trans
        rospy.loginfo("Pose: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(X, Y , eT,target_object))
        if abs(eX) <=0.05 :
            print ('here')
            eX = 0
        if abs(eY) <=0.05  :
            eY = 0
        if abs(eT   ) < 0.1:
            eT = 0
        succ =  eX == 0 and eY == 0 and eT==0            
        omni_base.tiny_move( velX=0.2*+eX,velY=0.3*eY, velT=-eT,std_time=0.2, MAX_VEL=0.3) 

    

In [ ]:
tf_man.getTF('placing_area')

In [ ]:
base_grasp_D(tf_name='placing_area')

In [ ]:
base_grasp_D(tf_name='placing_area',d_x=0.76, d_y=0.0,timeout=30)

In [ ]:
gripper.open()

In [ ]:
target_object='placing_area'
_,rot= tf_man.getTF("base_link",ref_frame='map')
trans,_=tf_man.getTF(target_object,ref_frame="base_link")

 
eX, eY, eZ = trans
eT= tf.transformations.euler_from_quaternion(rot)[2] - original_rot #Original 
print (eT)
if eT > np.pi: eT=-2*np.pi+eT
if eT < -np.pi: eT= 2*np.pi+eT
rospy.loginfo("error: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(eX, eY , eT,target_object))
X, Y, Z = trans
rospy.loginfo("Pose: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(X, Y , eT,target_object))

In [ ]:
omni_base.tiny_move( velX=0.1*+eX,velY=0.1*eY, velT=-eT,std_time=0.2, MAX_VEL=0.3) 

In [ ]:
d_x=0.58
d_y= 0.1
_,rot= tf_man.getTF("base_link",ref_frame='map')
trans,_=tf_man.getTF(target_object,ref_frame="base_link")
#trans
eX, eY, eZ = trans
eX+= -d_x  #x offest
eY+= -d_y #y Offset
eT= tf.transformations.euler_from_quaternion(rot)[2] - original_rot #Original 
print (eT)
if eT > np.pi: eT=-2*np.pi+eT
if eT < -np.pi: eT= 2*np.pi+eT
rospy.loginfo("error: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(eX, eY , eT,target_object))
X, Y, Z = trans
rospy.loginfo("Pose: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(X, Y , eT,target_object))
if abs(eX) <=0.05 :
    print ('here')
    eX = 0
if abs(eY) <=0.05  :
    eY = 0
if abs(eT   ) < 0.1:
    eT = 0
succ =  eX == 0 and eY == 0 and eT==0      
succ
#omni_base.tiny_move( velX=0.2*+eX,velY=0.3*eY, velT=-eT,std_time=0.2, MAX_VEL=0.3) 


In [ ]:
omni_base.tiny_move( velX=0.1*+eX,velY=0.1*eY, velT=-eT,std_time=0.2, MAX_VEL=0.3) 

In [ ]:
arm.get_current_pose()

In [ ]:
av=arm.get_current_joint_values()
av[0]+=-0.1
arm.go(av)

In [ ]:
arm.set_named_target('go')
arm.go()

In [ ]:
omni_base.move_base('place_shelf')

In [ ]:
whole_body.allow_replanning(True)
whole_body.set_num_planning_attempts(10)
whole_body.set_planning_time(10.0)
whole_body.set_workspace([-2.0, -2.0, 0.0, 2.0, 2.0, 2.0])

In [ ]:
object_point = PointStamped()
object_point

In [ ]:
target_position=[1.0,1.0,1.0]
object_point = PointStamped()
object_point.header.frame_id = "base_link"
object_point.point.x = target_position[0]
object_point.point.y = target_position[1]
object_point.point.z = target_position[2]
#transformar la posicion del objeto al marco de referencia de la base del robot

try:
    transformed_object_point = tfBuffer.transform(object_point, "base_link", timeout=rospy.Duration(1))
    transformed_base = self.tfBuffer.lookup_transform("odom", "base_link", rospy.Time(0), timeout=rospy.Duration(1))
except :
    rospy.WARN("Error al transformar la posicion del objeto al marco de referencia")


In [ ]:
tfBuffer.transform(object_point,'base_link',timeout=rospy.Duration(3))

In [ ]:
whole_body.set_start_state_to_current_state()
whole_body.set_position_target(position_goal)
succ = whole_body.go(wait= True)
rospy.sleep(0.5)
whole_body.stop()

In [ ]:
tfBuffer = tf2_ros.Buffer()


In [2]:
omni_base.move_base(known_location='pickup', time_out=200)

header: 
  seq: 0
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: ''
goal_id: 
  stamp: 
    secs: 0
    nsecs:         0
  id: ''
goal: 
  x: 0.0
  y: 0.0
  yaw: 0.0
  timeout: 200
  known_location: "pickup"


[ WARN] [1711575112.089443676]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1554.423000 according to authority /pose_integrator
[ WARN] [1711575112.090012157]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1554.423000 according to authority /pose_integrator
[ WARN] [1711575112.090068970]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1554.423000 according to authority /pose_integrator
[ WARN] [1711575112.090106398]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1554.423000 according to authority /pose_integrator
[ WARN] [1711575112.090146694]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1554.423000 according to authority /pose_integrator
[ WARN] [1711575112.090190297]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1554.423000 according to authority /pose_integrator
[ WARN] [1711575118.26

3

[ WARN] [1711575140.033341953]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1565.287000 according to authority /pose_integrator
[ WARN] [1711575140.033379734]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1565.287000 according to authority /pose_integrator
[ WARN] [1711575140.033406204]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1565.287000 according to authority /pose_integrator
[ WARN] [1711575140.033431125]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1565.287000 according to authority /pose_integrator
[ WARN] [1711575140.033454497]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1565.287000 according to authority /pose_integrator
[ WARN] [1711575140.033478474]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1565.287000 according to authority /pose_integrator
[ WARN] [1711575143.37

In [3]:
head.set_joint_values([ 0.0, -0.5])


[ WARN] [1711575150.576253491]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1569.752000 according to authority /pose_integrator
[ WARN] [1711575150.576310659]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1569.752000 according to authority /pose_integrator
[ WARN] [1711575150.576358499]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1569.752000 according to authority /pose_integrator
[ WARN] [1711575150.576394030]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1569.752000 according to authority /pose_integrator
[ WARN] [1711575150.576432223]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1569.752000 according to authority /pose_integrator
[ WARN] [1711575150.576470241]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1569.752000 according to authority /pose_integrator
[ WARN] [1711575151.46

In [4]:
img_msg  = bridge.cv2_to_imgmsg( cv2.cvtColor(rgbd.get_image(), cv2.COLOR_RGB2BGR))### GAZEBO BGR!?!??!
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
objects=detect_object_yolo('all',res)   

[ WARN] [1711575153.621789211]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1571.054000 according to authority /pose_integrator
[ WARN] [1711575153.621852564]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1571.054000 according to authority /pose_integrator
[ WARN] [1711575153.621895980]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1571.054000 according to authority /pose_integrator
[ WARN] [1711575153.621930449]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1571.054000 according to authority /pose_integrator
[ WARN] [1711575153.621961457]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1571.054000 according to authority /pose_integrator
[ WARN] [1711575153.622002931]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1571.054000 according to authority /pose_integrator


In [5]:
rospack = rospkg.RosPack()
file_path = rospack.get_path('config_files') 
objs = pd.read_csv (file_path+'/objects.csv')
objs=objs.drop(columns='Unnamed: 0')
print (objs)

Empty DataFrame
Columns: [x, y, z, obj_name]
Index: []


In [6]:
def check_if_grasped(pose_target,test_pt,tolerance=0.05):    
    return np.linalg.norm(pose_target-obj_pt)<tolerance

In [7]:
np.linalg.norm(pose_target-obj_pt)

NameError: name 'pose_target' is not defined

In [8]:
test_pt=objs[objs['obj_name']=='orange'][['x','y','z']].values
check_if_grasped(pose_target,test_pt)

NameError: name 'pose_target' is not defined

In [15]:
def check_if_grasped(pose_target,test_pt,tolerance=0.05):return np.linalg.norm(pose_target-test_pt)<tolerance


##############################
pose_target,_=tf_man.getTF('plum')
#########################

if len (objects)!=0 :
    for i in range(len(res.poses)):
        
        position = [res.poses[i].position.x ,res.poses[i].position.y,res.poses[i].position.z]
        
        object_point = PointStamped()
        object_point.header.frame_id = "head_rgbd_sensor_rgb_frame"
        object_point.point.x = position[0]
        object_point.point.y = position[1]
        object_point.point.z = position[2]
        position_map = tfBuffer.transform(object_point, "map", timeout=rospy.Duration(1))
        
        tf_man.pub_static_tf(pos= [position_map.point.x,position_map.point.y,position_map.point.z], rot=[0,0,0,1], ref="map", point_name=res.names[i].data[4:] )
        test_pt=np.asarray((position_map.point.x,position_map.point.y,position_map.point.z))
        print (np.linalg.norm(pose_target-test_pt))
        if check_if_grasped(pose_target,test_pt):
            print (f'Centroid found in area {test_pt}, obj_name: {res.names[i].data[4:]}')
            print ('Grasping May have failed')
        
        
        
        
        

0.056519773702052925
0.08126544138618978
0.3015221335441748
0.3062697417857679
2.338538791720599
2.2166820589117058
2.420777855332944


[ WARN] [1711575421.797319997]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1682.514000 according to authority /pose_integrator
[ WARN] [1711575421.797366832]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1682.514000 according to authority /pose_integrator
[ WARN] [1711575421.797393643]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1682.514000 according to authority /pose_integrator
[ WARN] [1711575421.797419227]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1682.514000 according to authority /pose_integrator
[ WARN] [1711575421.797444659]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1682.514000 according to authority /pose_integrator
[ WARN] [1711575421.797481911]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1682.514000 according to authority /pose_integrator
[ WARN] [1711575424.21

[ WARN] [1711575452.744150630]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1695.285000 according to authority /pose_integrator
[ WARN] [1711575452.744210622]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1695.285000 according to authority /pose_integrator
[ WARN] [1711575452.744255274]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1695.285000 according to authority /pose_integrator
[ WARN] [1711575452.744301294]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1695.285000 according to authority /pose_integrator
[ WARN] [1711575452.744350136]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1695.285000 according to authority /pose_integrator
[ WARN] [1711575452.744387835]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1695.285000 according to authority /pose_integrator
[ WARN] [1711575456.69

[ WARN] [1711575477.817024951]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1705.701000 according to authority /pose_integrator
[ WARN] [1711575477.817109373]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1705.701000 according to authority /pose_integrator
[ WARN] [1711575477.817166190]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1705.701000 according to authority /pose_integrator
[ WARN] [1711575477.817224909]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1705.701000 according to authority /pose_integrator
[ WARN] [1711575477.817280018]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1705.701000 according to authority /pose_integrator
[ WARN] [1711575477.817337296]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1705.701000 according to authority /pose_integrator
[ WARN] [1711575482.50

[ WARN] [1711575495.741039873]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1713.361000 according to authority /pose_integrator
[ WARN] [1711575495.741104523]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1713.361000 according to authority /pose_integrator
[ WARN] [1711575495.741154665]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1713.361000 according to authority /pose_integrator
[ WARN] [1711575495.741194412]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1713.361000 according to authority /pose_integrator
[ WARN] [1711575495.741229275]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1713.361000 according to authority /pose_integrator
[ WARN] [1711575495.741263700]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 1713.361000 according to authority /pose_integrator
[ WARN] [1711575497.60

...

[ WARN] [1711576485.734715351]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2134.937000 according to authority /pose_integrator
[ WARN] [1711576485.734766175]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2134.937000 according to authority /pose_integrator
[ WARN] [1711576485.734805150]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2134.937000 according to authority /pose_integrator
[ WARN] [1711576485.734845576]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2134.937000 according to authority /pose_integrator
[ WARN] [1711576485.734885245]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2134.937000 according to authority /pose_integrator
[ WARN] [1711576485.734923959]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2134.937000 according to authority /pose_integrator
[ WARN] [1711576488.72

[ WARN] [1711576497.216158752]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2139.942000 according to authority /pose_integrator
[ WARN] [1711576497.216221059]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2139.942000 according to authority /pose_integrator
[ WARN] [1711576497.216272585]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2139.942000 according to authority /pose_integrator
[ WARN] [1711576497.216321064]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2139.942000 according to authority /pose_integrator
[ WARN] [1711576497.216408964]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2139.942000 according to authority /pose_integrator
[ WARN] [1711576497.216471521]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2139.942000 according to authority /pose_integrator
[ WARN] [1711576501.88

[ WARN] [1711576510.046060368]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2145.406000 according to authority /pose_integrator
[ WARN] [1711576510.046114533]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2145.406000 according to authority /pose_integrator
[ WARN] [1711576510.046148579]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2145.406000 according to authority /pose_integrator
[ WARN] [1711576510.046181477]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2145.406000 according to authority /pose_integrator
[ WARN] [1711576510.046213734]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2145.406000 according to authority /pose_integrator
[ WARN] [1711576510.046249517]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2145.406000 according to authority /pose_integrator
[ WARN] [1711576510.96

[ WARN] [1711576516.809165358]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2148.522000 according to authority /pose_integrator
[ WARN] [1711576516.809227166]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2148.522000 according to authority /pose_integrator
[ WARN] [1711576516.809273245]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2148.522000 according to authority /pose_integrator
[ WARN] [1711576516.809313586]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2148.522000 according to authority /pose_integrator
[ WARN] [1711576516.809354266]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2148.522000 according to authority /pose_integrator
[ WARN] [1711576516.809390130]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2148.522000 according to authority /pose_integrator
[ WARN] [1711576517.19

[ WARN] [1711576535.228656665]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2156.424000 according to authority /pose_integrator
[ WARN] [1711576535.228714988]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2156.424000 according to authority /pose_integrator
[ WARN] [1711576535.228757526]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2156.424000 according to authority /pose_integrator
[ WARN] [1711576535.228799484]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2156.424000 according to authority /pose_integrator
[ WARN] [1711576535.228838055]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2156.424000 according to authority /pose_integrator
[ WARN] [1711576535.228875398]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2156.424000 according to authority /pose_integrator
[ WARN] [1711576536.05

In [10]:

if len (objects)!=0 :
    for i in range(len(res.poses)):
        #tf_man.getTF("head_rgbd_sensor_rgb_frame")
        position = [res.poses[i].position.x ,res.poses[i].position.y,res.poses[i].position.z]
        print ('position,name',position,res.names[i].data[4:])
        ##########################################################

        object_point = PointStamped()
        object_point.header.frame_id = "head_rgbd_sensor_rgb_frame"
        object_point.point.x = position[0]
        object_point.point.y = position[1]
        object_point.point.z = position[2]
        position_map = tfBuffer.transform(object_point, "map", timeout=rospy.Duration(1))
        
        
        
        print ('position_map',position_map)
        tf_man.pub_static_tf(pos= [position_map.point.x,position_map.point.y,position_map.point.z], rot=[0,0,0,1], ref="map", point_name=res.names[i].data[4:] )
        new_row = {'x': position_map.point.x, 'y': position_map.point.y, 'z': position_map.point.z, 'obj_name': res.names[i].data[4:]}
        objs.loc[len(objs)] = new_row
        ###########################################################

position,name [-0.3964172303676605, 0.0009068404324352741, 1.1475542783737183] plum
position_map header: 
  seq: 0
  stamp: 
    secs: 1594
    nsecs: 501000000
  frame_id: "map"
point: 
  x: 6.1156200123683435
  y: 4.620429126536116
  z: 0.4279542371487296
position,name [-0.26395922899246216, -0.018268687650561333, 1.1705325841903687] orange
position_map header: 
  seq: 0
  stamp: 
    secs: 1594
    nsecs: 501000000
  frame_id: "map"
point: 
  x: 6.25116492614787
  y: 4.6263182192908365
  z: 0.4337611708322301
position,name [-0.08879821747541428, -0.0209287628531456, 1.3269072771072388] spoon
position_map header: 
  seq: 0
  stamp: 
    secs: 1594
    nsecs: 501000000
  frame_id: "map"
point: 
  x: 6.447732514584939
  y: 4.732259113636191
  z: 0.3611029942658902
position,name [-0.6469786763191223, -0.05441746115684509, 1.1909737586975098] cracker_box
position_map header: 
  seq: 0
  stamp: 
    secs: 1594
    nsecs: 501000000
  frame_id: "map"
point: 
  x: 5.88010693250541
  y: 4.727

In [ ]:
head.to_tf('plum')

In [ ]:
base_grasp_D('plum',timeout=30)

In [ ]:
objs['x']

In [ ]:
obj_pt=np.asarray((position_map.point.x,
position_map.point.y,
position_map.point.z))

if np.linalg.norm(pose_target-obj_pt)<0.05:print ('I say chale man')

In [ ]:
pose_target,_=tf_man.getTF('plum')

In [ ]:
objs['x']

In [ ]:

np.linalg.norm(pose_target-obj_pt)

In [ ]:
pose_target

In [ ]:
obj_pt

In [3]:
av=arm.get_current_joint_values()
av

[0.1398872533758338,
 -0.36002252801888535,
 -0.05070234534369389,
 -0.9706266302849658,
 0.00046110822381617567,
 0.0]

In [5]:
brazo.set_joint_values(av[:-1])

True

shutdown request: [/smach] Reason: new node registered with same name


[ERROR] [1711995010.714698255]: Ignoring transform for child_frame_id "windex_bottle" from authority "unknown_publisher" because of a nan value in the transform (-nan -nan -nan) (0.000000 0.000000 0.000000 1.000000)
[ERROR] [1711995010.745141780]: Ignoring transform for child_frame_id "medium_clamp" from authority "unknown_publisher" because of a nan value in the transform (-nan -nan -nan) (0.000000 0.000000 0.000000 1.000000)
[ERROR] [1711995278.640385096]: Ignoring transform for child_frame_id "windex_bottle" from authority "unknown_publisher" because of a nan value in the transform (-nan -nan -nan) (0.000000 0.000000 0.000000 1.000000)
[ERROR] [1711995280.489316251]: Ignoring transform for child_frame_id "windex_bottle" from authority "unknown_publisher" because of a nan value in the transform (-nan -nan -nan) (0.000000 0.000000 0.000000 1.000000)
[ERROR] [1711995337.267017848]: Ignoring transform for child_frame_id "sugar_box" from authority "unknown_publisher" because of a nan val

In [8]:
av= arm.get_current_joint_values()

In [9]:
av

[0.6200031129804957,
 -1.0618195280188854,
 -9.345343694011632e-06,
 -1.7499976302849658,
 -0.0003068917761841483,
 0.0]